<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/evaluation/Retrievals_approaches_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 14.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.73-cp310-cp310-linux_x86_64.whl size=61754255 sha256=2dd74cfaaf5804f890c9f2fde646c93faca5f8bdef7a27eb79d0d4db1cb279eb
  Stored in directory: /root/.cache/pip/wheels/f0/96/43/7e194c954ac912d68301956136f2b931efadf8f56c71fd5400
Successfully built llama-cpp-python


In [2]:
!huggingface-cli download microsoft/Phi-3-mini-4k-instruct-gguf Phi-3-mini-4k-instruct-fp16.gguf --local-dir ./models --local-dir-use-symlinks False

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf to /root/.cache/huggingface/hub/tmplxlp382v
Phi-3-mini-4k-instruct-fp16.gguf: 100% 7.64G/7.64G [01:05<00:00, 117MB/s]
./models/Phi-3-mini-4k-instruct-fp16.gguf


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install langchain langchain-core langchain-community --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 17.2 MB/s eta 0:00:00


In [5]:
from langchain_community.llms import LlamaCpp
TEMP = 0
N_CTX = 4096
N_GPU_L = -1

llm = LlamaCpp(
    model_path="/content/models/Phi-3-mini-4k-instruct-fp16.gguf",
    temperature=TEMP,
    n_ctx=N_CTX,
    n_gpu_layers = N_GPU_L,
    verbose=True
)

llama_model_loader: loaded meta data with 23 key-value pairs and 195 tensors from /content/models/Phi-3-mini-4k-instruct-fp16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.attention.head_count

In [6]:
### Retrieval Grader

from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp
from langchain_core.output_parsers import JsonOutputParser

prompt = PromptTemplate(
    template="""
    <|assistant|> You are a grader assessing relevance of a retrieved document to a user question. If the document contains keywords related to the user question, grade it as relevant.
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explaination. <|end|>
    <|user|> Here is the retrieved document: \n\n {document} \n\n Here is the user question: {question} <|end|>
    <|assistant|>
    """,
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()

In [7]:
import pandas as pd
app1 = pd.read_json('/content/drive/MyDrive/Thesis/retrievers/results_approach1.json')
len(app1)

64

In [8]:
results_list = []
for i in range(len(app1)):
  question = app1['question'][i]
  doc_txt = app1['context_cosine'][i]['page_content']
  answer = retrieval_grader.invoke({"question": question, "document": doc_txt})
  results_list.append(pd.DataFrame({'question_name': question, 'answer': [answer]}))
results = pd.concat(results_list, ignore_index=True)


llama_print_timings:        load time =     412.01 ms
llama_print_timings:      sample time =       4.21 ms /     7 runs   (    0.60 ms per token,  1662.31 tokens per second)
llama_print_timings: prompt eval time =    4565.69 ms /   792 tokens (    5.76 ms per token,   173.47 tokens per second)
llama_print_timings:        eval time =     336.35 ms /     6 runs   (   56.06 ms per token,    17.84 tokens per second)
llama_print_timings:       total time =    5398.24 ms /   798 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     412.01 ms
llama_print_timings:      sample time =       4.79 ms /     7 runs   (    0.68 ms per token,  1462.60 tokens per second)
llama_print_timings: prompt eval time =    3785.42 ms /   613 tokens (    6.18 ms per token,   161.94 tokens per second)
llama_print_timings:        eval time =     234.74 ms /     6 runs   (   39.12 ms per token,    25.56 tokens per second)
llama_print_timings:       total time =    4723.78 ms /   619 

In [9]:
score = 0
for i in range(len(results)):
  if results['answer'][i]['score'] == 'yes':
    score += 1
total_score_app1 = score/len(results)
print(f'Total score for Approach # 1 is {total_score_app1}')

Total score for Approach # 1 is 0.703125


In [10]:
app2 = pd.read_json('/content/drive/MyDrive/Thesis/retrievers/results_approach2.json')
len(app2)

66

In [15]:
from json import JSONDecodeError
from langchain_core.exceptions import OutputParserException

In [16]:
results_list = []
for i in range(len(app2)):
  question = app2['question_name'][i]
  doc_txt = app2['context_cosine_name'][i]['page_content']
  try:
    answer = retrieval_grader.invoke({"question": question, "document": doc_txt})
    results_list.append(pd.DataFrame({'question_name': question, 'answer': [answer]}))
  except JSONDecodeError as e:
    print(f"JSONDecodeError occurred for question: {question}. Skipping...")
    continue
  except OutputParserException as e:
    print(f"OutputParserException occurred for question: {question}. Skipping...")
    continue
results = pd.concat(results_list, ignore_index=True)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     412.01 ms
llama_print_timings:      sample time =       3.62 ms /     6 runs   (    0.60 ms per token,  1657.00 tokens per second)
llama_print_timings: prompt eval time =    6070.49 ms /  1083 tokens (    5.61 ms per token,   178.40 tokens per second)
llama_print_timings:        eval time =     203.38 ms /     5 runs   (   40.68 ms per token,    24.58 tokens per second)
llama_print_timings:       total time =    6689.56 ms /  1088 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     412.01 ms
llama_print_timings:      sample time =       3.51 ms /     7 runs   (    0.50 ms per token,  1992.03 tokens per second)
llama_print_timings: prompt eval time =    5093.22 ms /   926 tokens (    5.50 ms per token,   181.81 tokens per second)
llama_print_timings:        eval time =     243.68 ms /     6 runs   (   40.61 ms per token,    24.62 tokens per second)
llama_print_timings:       to

OutputParserException occurred for question: Are there any tax-related risks or benefits for the COCA COLA CO mentioned?. Skipping...



llama_print_timings:        load time =     412.01 ms
llama_print_timings:      sample time =       3.08 ms /     6 runs   (    0.51 ms per token,  1949.32 tokens per second)
llama_print_timings: prompt eval time =    3342.80 ms /   629 tokens (    5.31 ms per token,   188.17 tokens per second)
llama_print_timings:        eval time =     193.61 ms /     5 runs   (   38.72 ms per token,    25.83 tokens per second)
llama_print_timings:       total time =    3706.30 ms /   634 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     412.01 ms
llama_print_timings:      sample time =       4.70 ms /     9 runs   (    0.52 ms per token,  1914.49 tokens per second)
llama_print_timings: prompt eval time =    3962.08 ms /   735 tokens (    5.39 ms per token,   185.51 tokens per second)
llama_print_timings:        eval time =     313.97 ms /     8 runs   (   39.25 ms per token,    25.48 tokens per second)
llama_print_timings:       total time =    4608.40 ms /   743 

In [20]:
score = 0
for i in range(len(results)):
  if (results['answer'][i] is not None) and (results['answer'][i]['score'] == 'yes'):
    score += 1
total_score_app2 = score/len(results)
print(f'Total score for Approach # 2 is {total_score_app2}')

Total score for Approach # 2 is 0.6615384615384615


In [21]:
app3 = pd.read_json('/content/drive/MyDrive/Thesis/retrievers/results_approach3.json')
len(app3)

64

In [22]:
results_list = []
for i in range(len(app3)):
  question = app3['question'][i]
  doc_txt = app3['context_ip'][i]['page_content']
  answer = retrieval_grader.invoke({"question": question, "document": doc_txt})
  results_list.append(pd.DataFrame({'question_name': question, 'answer': [answer]}))
results = pd.concat(results_list, ignore_index=True)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     412.01 ms
llama_print_timings:      sample time =       3.24 ms /     7 runs   (    0.46 ms per token,  2161.16 tokens per second)
llama_print_timings: prompt eval time =    3589.35 ms /   659 tokens (    5.45 ms per token,   183.60 tokens per second)
llama_print_timings:        eval time =     236.73 ms /     6 runs   (   39.45 ms per token,    25.35 tokens per second)
llama_print_timings:       total time =    4092.24 ms /   665 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     412.01 ms
llama_print_timings:      sample time =       3.91 ms /     7 runs   (    0.56 ms per token,  1792.57 tokens per second)
llama_print_timings: prompt eval time =    3336.90 ms /   613 tokens (    5.44 ms per token,   183.70 tokens per second)
llama_print_timings:        eval time =     233.04 ms /     6 runs   (   38.84 ms per token,    25.75 tokens per second)
llama_print_timings:       to

In [23]:
score = 0
for i in range(len(results)):
  if results['answer'][i]['score'] == 'yes':
    score += 1
total_score_app3 = score/len(results)
print(f'Total score for Approach # 3 is {total_score_app3}')

Total score for Approach # 3 is 0.703125


In [24]:
app4 = pd.read_json('/content/drive/MyDrive/Thesis/retrievers/results_approach4.json')
len(app4)

66

In [26]:
results_list = []
for i in range(len(app4)):
  question = app4['question_name'][i]
  doc_txt = app4['context_ip_name'][i]['page_content']
  try:
    answer = retrieval_grader.invoke({"question": question, "document": doc_txt})
    results_list.append(pd.DataFrame({'question_name': question, 'answer': [answer]}))
  except JSONDecodeError as e:
    print(f"JSONDecodeError occurred for question: {question}. Skipping...")
    continue
  except OutputParserException as e:
    print(f"OutputParserException occurred for question: {question}. Skipping...")
    continue
  results_list.append(pd.DataFrame({'question_name': question, 'answer': [answer]}))
results = pd.concat(results_list, ignore_index=True)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     412.01 ms
llama_print_timings:      sample time =       2.91 ms /     6 runs   (    0.48 ms per token,  2065.40 tokens per second)
llama_print_timings: prompt eval time =    6022.87 ms /  1083 tokens (    5.56 ms per token,   179.81 tokens per second)
llama_print_timings:        eval time =     201.22 ms /     5 runs   (   40.24 ms per token,    24.85 tokens per second)
llama_print_timings:       total time =    6520.75 ms /  1088 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     412.01 ms
llama_print_timings:      sample time =       3.94 ms /     7 runs   (    0.56 ms per token,  1778.00 tokens per second)
llama_print_timings: prompt eval time =    5078.64 ms /   926 tokens (    5.48 ms per token,   182.33 tokens per second)
llama_print_timings:        eval time =     242.84 ms /     6 runs   (   40.47 ms per token,    24.71 tokens per second)
llama_print_timings:       to

OutputParserException occurred for question: Are there any tax-related risks or benefits for the COCA COLA CO mentioned?. Skipping...



llama_print_timings:        load time =     412.01 ms
llama_print_timings:      sample time =       3.36 ms /     6 runs   (    0.56 ms per token,  1785.18 tokens per second)
llama_print_timings: prompt eval time =    3341.14 ms /   629 tokens (    5.31 ms per token,   188.26 tokens per second)
llama_print_timings:        eval time =     192.39 ms /     5 runs   (   38.48 ms per token,    25.99 tokens per second)
llama_print_timings:       total time =    3708.22 ms /   634 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     412.01 ms
llama_print_timings:      sample time =       5.46 ms /     9 runs   (    0.61 ms per token,  1649.26 tokens per second)
llama_print_timings: prompt eval time =    3936.57 ms /   735 tokens (    5.36 ms per token,   186.71 tokens per second)
llama_print_timings:        eval time =     317.11 ms /     8 runs   (   39.64 ms per token,    25.23 tokens per second)
llama_print_timings:       total time =    4471.88 ms /   743 

In [28]:
score = 0
for i in range(len(results)):
  if (results['answer'][i] is not None) and (results['answer'][i]['score'] == 'yes'):
    score += 1
total_score_app4 = score/len(results)
print(f'Total score for Approach # 4 is {total_score_app4}')

Total score for Approach # 4 is 0.6615384615384615


In [29]:
app5 = pd.read_json('/content/drive/MyDrive/Thesis/retrievers/results_approach5.json')
len(app5)

64

In [30]:
results_list = []
for i in range(len(app5)):
  question = app5['question'][i]
  doc_txt = app5['context_l2'][i]['page_content']
  answer = retrieval_grader.invoke({"question": question, "document": doc_txt})
  results_list.append(pd.DataFrame({'question_name': question, 'answer': [answer]}))
results = pd.concat(results_list, ignore_index=True)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     412.01 ms
llama_print_timings:      sample time =       3.30 ms /     7 runs   (    0.47 ms per token,  2121.86 tokens per second)
llama_print_timings: prompt eval time =    3570.87 ms /   659 tokens (    5.42 ms per token,   184.55 tokens per second)
llama_print_timings:        eval time =     235.41 ms /     6 runs   (   39.24 ms per token,    25.49 tokens per second)
llama_print_timings:       total time =    4023.16 ms /   665 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     412.01 ms
llama_print_timings:      sample time =       3.80 ms /     7 runs   (    0.54 ms per token,  1841.14 tokens per second)
llama_print_timings: prompt eval time =    3263.85 ms /   613 tokens (    5.32 ms per token,   187.81 tokens per second)
llama_print_timings:        eval time =     233.52 ms /     6 runs   (   38.92 ms per token,    25.69 tokens per second)
llama_print_timings:       to

In [31]:
score = 0
for i in range(len(results)):
  if results['answer'][i]['score'] == 'yes':
    score += 1
total_score_app5 = score/len(results)
print(f'Total score for Approach # 5 is {total_score_app5}')

Total score for Approach # 5 is 0.703125


In [32]:
app6 = pd.read_json('/content/drive/MyDrive/Thesis/retrievers/results_approach6.json')
len(app6)

66

In [34]:
results_list = []
for i in range(len(app6)):
  question = app6['question_name'][i]
  doc_txt = app6['context_ip_name'][i]['page_content']
  try:
    answer = retrieval_grader.invoke({"question": question, "document": doc_txt})
    results_list.append(pd.DataFrame({'question_name': question, 'answer': [answer]}))
  except JSONDecodeError as e:
    print(f"JSONDecodeError occurred for question: {question}. Skipping...")
    continue
  except OutputParserException as e:
    print(f"OutputParserException occurred for question: {question}. Skipping...")
    continue
results = pd.concat(results_list, ignore_index=True)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     412.01 ms
llama_print_timings:      sample time =       2.89 ms /     6 runs   (    0.48 ms per token,  2079.00 tokens per second)
llama_print_timings: prompt eval time =    6047.28 ms /  1083 tokens (    5.58 ms per token,   179.09 tokens per second)
llama_print_timings:        eval time =     202.07 ms /     5 runs   (   40.41 ms per token,    24.74 tokens per second)
llama_print_timings:       total time =    6697.32 ms /  1088 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     412.01 ms
llama_print_timings:      sample time =       3.56 ms /     7 runs   (    0.51 ms per token,  1965.74 tokens per second)
llama_print_timings: prompt eval time =    5063.39 ms /   926 tokens (    5.47 ms per token,   182.88 tokens per second)
llama_print_timings:        eval time =     243.18 ms /     6 runs   (   40.53 ms per token,    24.67 tokens per second)
llama_print_timings:       to

OutputParserException occurred for question: Are there any tax-related risks or benefits for the COCA COLA CO mentioned?. Skipping...



llama_print_timings:        load time =     412.01 ms
llama_print_timings:      sample time =       3.99 ms /     6 runs   (    0.66 ms per token,  1504.51 tokens per second)
llama_print_timings: prompt eval time =    3350.80 ms /   629 tokens (    5.33 ms per token,   187.72 tokens per second)
llama_print_timings:        eval time =     195.73 ms /     5 runs   (   39.15 ms per token,    25.55 tokens per second)
llama_print_timings:       total time =    3769.27 ms /   634 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     412.01 ms
llama_print_timings:      sample time =       4.56 ms /     9 runs   (    0.51 ms per token,  1972.39 tokens per second)
llama_print_timings: prompt eval time =    3957.98 ms /   735 tokens (    5.39 ms per token,   185.70 tokens per second)
llama_print_timings:        eval time =     314.66 ms /     8 runs   (   39.33 ms per token,    25.42 tokens per second)
llama_print_timings:       total time =    4566.19 ms /   743 

In [35]:
score = 0
for i in range(len(results)):
  if (results['answer'][i] is not None) and (results['answer'][i]['score'] == 'yes'):
    score += 1
total_score_app6 = score/len(results)
print(f'Total score for Approach # 6 is {total_score_app6}')

Total score for Approach # 6 is 0.6615384615384615


# Final results:

In [36]:
print(f'Total score for Approach # 1 is {total_score_app1}')
print(f'Total score for Approach # 2 is {total_score_app2}')
print(f'Total score for Approach # 3 is {total_score_app3}')
print(f'Total score for Approach # 4 is {total_score_app4}')
print(f'Total score for Approach # 5 is {total_score_app5}')
print(f'Total score for Approach # 6 is {total_score_app6}')

Total score for Approach # 1 is 0.703125
Total score for Approach # 2 is 0.6615384615384615
Total score for Approach # 3 is 0.703125
Total score for Approach # 4 is 0.6615384615384615
Total score for Approach # 5 is 0.703125
Total score for Approach # 6 is 0.6615384615384615
